In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import time
import re
import urllib
import pandas as pd
import seaborn as sns
from requests.exceptions import HTTPError

In [9]:
ultimate_data = pd.read_csv('draftsamples.csv')
ultimate_data = ultimate_data.rename(columns={"Unnamed: 0" : "TeamID"})
ultimate_data = ultimate_data.set_index("TeamID")
ultimate_data = ultimate_data.drop("07", axis=1)
ultimate_data = ultimate_data.replace('NaN', np.nan).dropna()
ultimate_data = ultimate_data.replace(['DT', 'DE', 'LB', 'S"', 'PK'], np.nan)
ultimate_data = ultimate_data.dropna()
ultimate_data = ultimate_data.drop("League", axis=1)
ultimate_data

,01,02,03,04,05,06,Points
TeamID,,,,,,,
1201310003,RB,TE,RB,RB,QB,WR,0.658787
6201310003,RB,WR,RB,RB,QB,WR,-0.160545
3201010005,QB,RB,RB,WR,WR,WR,-1.264911
10201210005,RB,QB,WR,WR,TE,RB,0.325290
7201310005,RB,TE,WR,WR,WR,QB,-1.017693
9201310005,RB,RB,RB,WR,RB,QB,-1.017693
3201510005,RB,WR,QB,RB,WR,TE,-0.465549
4201510005,RB,WR,TE,QB,RB,WR,-0.465549
7201510005,RB,RB,WR,WR,QB,RB,-0.465549


In [10]:
two_rounds = ultimate_data.groupby(["01", "02"]).agg(['mean', 'count'])
two_rounds = two_rounds[two_rounds["Points", "count"] > 20]
two_rounds

Points      
           mean count
01 02                
QB RB -0.115459    82
   WR -0.213690    56
RB QB -0.207175    84
   RB  0.152498   227
   TE  0.199599    30
   WR  0.002790   377
TE RB  0.510322    26
   WR  0.022784    21
WR QB  0.136605    49
   RB -0.020715   289
   TE -0.180670    37
   WR -0.017092   239

In [69]:
cleaned = two_rounds.unstack(level=0).dropna()["Points"]["mean"]
cleaned.scatter()

AttributeError: 'DataFrame' object has no attribute 'scatter'

In [49]:
sns.factorplot(

MultiIndex(levels=[['Points'], ['mean', 'count']],
           labels=[[0, 0], [0, 1]])